In [1]:
import sys
import os
import numpy as np
import traceback

import sys 
sys.path.append("..")


In [2]:
import numpy as np
import matplotlib.pyplot as plt
from dataclasses import dataclass 
from physics.fom import sensitivity, sfm, mcf, tsf 
from physics.reflectometry import _kz, reflectivity
from solvers.grid_search import GridSearchSolver
from problems.base1 import Bounds, SOISpec, CapSpec, MRL, Materials, Base1OptimizationProblem, SubstrateSpec, MRL
from data.materials_loader import load_base1_materials

In [3]:
# simple magnetic SLD model for now – tweak later if you want
def m_sld_from_x(x: float) -> float:
    return 1.0 * x  # placeholder linear scaling

materials = load_base1_materials("data.json", m_sld_from_x=m_sld_from_x)

sois = [
    SOISpec("SOI1", rho_n=2.0, thickness=100.0, sigma=3.0),
    SOISpec("SOI2", rho_n=1.0, thickness=80.0, sigma=3.0),
    SOISpec("SOI2", rho_n=3.0, thickness=100.0, sigma=3.0)
]

Q = np.linspace(0.005, 0.3, 400)

bounds_x = Bounds(0.0, 1)      # x_coti
bounds_d = Bounds(20.0, 300.0)   # d_mrl
bounds_cap = Bounds(5.0, 200.0)    # d_cap

# define problem: 

problem = Base1OptimizationProblem(
    materials=materials,
    soi_list=sois,
    q_grid=Q,
    bounds_x=bounds_x,
    bounds_d=bounds_d,
    bounds_cap=bounds_cap,
)


In [4]:
space = problem.search_space
print(f"Params name: {space.names}")
print(f"dimensions: {len(space)}")

theta_raw = space.sample(5)   # (5, 3)
theta = np.vstack([space.clip(row) for row in theta_raw])  # clip each row

print("Raw samples:\n", theta_raw)
print("Clipped samples:\n", theta)

x0 = space.unpack(theta[0])   # first sample as dict
x0

print("---------------------------")
solver = GridSearchSolver(problem, n_points=100)
res = solver.run(evals=10_000)
print(res)
print("----------------------------")

Params name: ['x_coti', 'd_mrl', 'd_cap', 'cap']
dimensions: 4
Raw samples:
 [[1.53471517e-02 1.93520652e+02 1.84322034e+02 2.00000000e+00]
 [5.84496906e-01 9.92474317e+01 1.48179376e+02 1.00000000e+00]
 [5.64925075e-01 6.36247014e+01 3.99080641e+01 2.00000000e+00]
 [4.91161608e-01 2.90648670e+02 1.51182193e+02 1.00000000e+00]
 [3.43321464e-01 2.59499524e+02 1.72075917e+02 1.00000000e+00]]
Clipped samples:
 [[1.53471517e-02 1.93520652e+02 1.84322034e+02 2.00000000e+00]
 [5.84496906e-01 9.92474317e+01 1.48179376e+02 1.00000000e+00]
 [5.64925075e-01 6.36247014e+01 3.99080641e+01 2.00000000e+00]
 [4.91161608e-01 2.90648670e+02 1.51182193e+02 1.00000000e+00]
 [3.43321464e-01 2.59499524e+02 1.72075917e+02 1.00000000e+00]]
---------------------------
=== RunResults ===
n_evals : 100000
y_best  : 0.11561963691242409
x_best  :
  x_coti     = 0.030303030303030304
  d_mrl      = 45.45454545454545
  d_cap      = 22.727272727272727
  cap        = Al2O3
history : 100000 entries
history (first/last)